# 1. Activate GPU and Install Dependencies

In [ ]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

In [ ]:
! pip install pyarrow==15.0.0
! pip install requests==2.32.2
! pip install transformers[torch]
! pip install accelerate -U

In [ ]:
# Install required libraries
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

In [ ]:
# Load data
# from datasets import load_dataset
# imdb = load_dataset("imdb")

#2. Preprocess data

In [ ]:
file_path = 'ImdbDataset.csv'

# Try different encodings if one doesn't work
try:
    imdb = pd.read_csv(file_path, encoding='latin1', delimiter=',')
except UnicodeDecodeError:
    try:
        imdb = pd.read_csv(file_path, encoding='cp1252', delimiter=',')
    except UnicodeDecodeError:
        imdb = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=',')
except pd.errors.ParserError:
    print("Parsing error encountered.")
    # Further handling or inspection
    raise

print(imdb.head())

In [ ]:
# Create a smaller training dataset for faster training times
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])
print(small_train_dataset[0])
print(small_test_dataset[0])

In [ ]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

In [ ]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Training the model

In [ ]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
# Define the evaluation metrics
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Compute the evaluation metrics
trainer.evaluate()

# 4. Analyzing new data with the model

In [ ]:
# Upload the model to the Hub
trainer.push_to_hub()